# Test dataloader

In [1]:
from dataloader import get_train_loader
from utils import INNER_SIZE, OUTER_SIZE

In [2]:
loader = get_train_loader(['./dataset/TrainDataset/'], batchsize=2, inner_size=INNER_SIZE, outer_size=OUTER_SIZE, num_workers=1)

In [4]:
res = next(iter(loader))